<a href="https://colab.research.google.com/github/Shailesh0209/x_tools_in_ds_dipoma-iitm/blob/main/X_W7_Design_your_output.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Design_your_output: Tools for specialized visualizations: network of actors

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Download the data
!rm *.tsv.gz
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la

In [ ]:
# Load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=False).set_index('tconst')[['titleType', 'primaryTitle', 'startYear']]
title.head()

In [ ]:
# Load the cast of each film
cast = pd.read_csv('title.principals.tsv.gz', sep='\t')[['tconst', 'nconst', 'category']]
# ONly consider actors, not directors, composers, etc.Shrinks data to about 40%
cast = cast[cast.category.isin({'actor', 'actress'})]
cast.head()

In [ ]:
# Only consider movies, not TV series, etc. Shrinks data to ~5%
movies = title[title['titleType'] == 'movie']
cast = cast[cast['tcons'].isin(movies.index)]
# This is what the network looks like
cast.head()

In [ ]:
# Restrict data to just a single region (e.g. IN, US, etc)
# This loads the region for each title
region = pd.read_csv('title.akas.tsv.gz', sep='\t', low_memory=False).set_index('titleId')['region']
region.value_counts().head(10)

In [ ]:
# Load the name data along with birth year
name = pd.read_csv('name.basics.tsv.gz', sep='\t', na_values='\\N', dtype={'birthYear': float}).set_index('nconst')[[
                            'primaryName', 'birthYear']]

In [ ]:
name.head()

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
def get_pairs(lang=None, min_acted=25, min_pairings=1):
    '''
    Returns an adjacency matrix and actor mappping of actor pairs where:
    -Each actor has acted in at least min_acted films
    -The twoactors have acted together in at least min_pairings films
    -And (optionally), belong to a region `lang` (IN, UN, etc)
    '''
    graph = cast
    if lang is not None:
        graph = graph[graph['tconst'].isin(region[region == lang].index)]
    name_freq = graph['nconst'].value_counts()
    top_names = name_freq[name_freq >= min_acted]
    top_actors = graph[graph['nconst'].isin(top_names.index)]

    p = top_actors.copy()
    p['title'] = p['tconst'].astype('category')
    p['name'] = p['name'].cat.codes.values
    data = np.ones(len(p), dtype='int')

    matrix = csr_matrix((data, (row, col)))
    square = matrix.T * matrix
    square.setdiag(0)
    square = square.tocoo()

    pairs = pd.DataFrame({
        'row': squre.row,
        'col': squre.col,
        'n': square.data
    })
    pairs = pairs[pairs.n >= min_pairings].reset_index(drop = True)
    return pairs, name.reindex(p['name'].cat.categories)

    def lookup(pairs, cat):
        pairs = pd.concat([
                           pairs,
                           cat.iloc[pairs.row].reset_index(drop=True),
                           cat.iloc[pairs.col].reset_index(drop=True),
        ], axis=1)
        pairs = pairs.drop(columns=['row', 'col'])
        pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
        return pairs.sort_values('count', ascending=False)

In [ ]:
pairs, cat = get_pairs(lang='IN', min_acted=10, min_pairings=3)

In [ ]:
pairs

In [ ]:
cat

In [ ]:
ForKumu = lookup(pairs, cat)
ForKumu

In [ ]:
ForKumu = ForKumu[['name1', 'name2', 'count']]
ForKumu = ForKumu.rename(columns={'name1': 'From',
                                  'name2': 'To',
                                  'count': 'Strength'})
ForKumu

In [ ]:
ForKumu.to_excel("pairs.xslx", index=False)

# Modern tools to simplify deep learning models- Cluster the network of actors


In [ ]:
!pip install scikit-network
import pandas as pd
import numpy as np
import sknetwork.clustering
import sknetwork.utils
from scipy.sparse import csr_matrix

In [ ]:
# Download the data 
!rm -f *.tsv.akas.tsv.gz
!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la

In [ ]:
# Load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=False).set_index('tconst')[['titleType', 'primaryTitle', 'startYear']]
title.head()

In [ ]:
# Load the cast of each film
cast = pd.read_csv('title.principals.tsv.gz', sep='\t')[['tconst', 'nconst', 'category']]
# Only consider actors, not directors, composers, etc. Shrinks data to about 40%
cast = cast[cast.category.isin({'actor', 'actress'})]
cast.head()

In [ ]:
# Only consider movies, not TV series, etc. Shrinks data to ~5%
movies = title[title['titleType'] == 'movie']
cast = cast[cast['tconst'].isin(movies.index)]
# This is what the network looks like
cast.head()

In [ ]:
# Explore the regions we have data for (e.g. IN, US, etc)
region  = pd.read_csv('title.akas.tsv.gz', sep='\t', low_memory=False).set_index('titleId')['region']
region.value_counts().head(10)

In [ ]:
# Load the name data along with birth year
name = pd.read_csv('name.basics.tsv.gz', sep='\t', na_values='\\N', dtype={'birthYear': float}).set_index('nconst')[['primaryName', 'birthYear']]
name_freq = cast['nconst'].value_counts()

In [ ]:
def get_pairs(lang=None, min_acted=25, min_pairings=4):
    '''Returns an adjacency matrix and actor mapping of actor pairs where:
    -Each actor has acted in at least min_acted films
    -The two actors have acted together in at least min_pairings films
    - And (optionally), belong to a region `lang` (IN, UN, etc)
    '''
    graph = cast
    if lang is Not None:
        graph = graph[graph['tconst'].isin(regionn[region = lang].index)]
    top_names = name_freq[name_freq >= min_acted]
    top_actors = graph[graph['nconst'].isin(top_names.index)]

    p = top_actors.copy()
    p['title'] = p['tcons'].astype('category')
    p['name'] = p['nconst'].astype('category')

    row = p['title'].cat.codes.values
    col = p['name'].cat.codes.values
    data = np.ones(len(p), dtype='int')

    matrix = csr_matrix((data, (row, col)))
    square = csr_matrix((data, (row, col)))
    squre.setdiag(0)
    squre = squre.tocoo()

    pairs = pd.DataFrame({
        'row': squre.row,
        'col': squre.col,
        'n': squre.data
    })
    pairs = pairs[pairs.n >= min_pairings].reset_index(drop=True)
    return pairs, name.reindex(p['name'].cat.categories)

def lookup(pairs, cat):
    pairs = pd.concat([
                       pairs,
                       cat.iloc[pairs.row].reset_index(drop=True),
                       cat.iloc[pairs.col].reset_index(drop=True),
    ], axis=1)
    pairs = pairs.drop(columns=['row', 'col'])
    pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
    return pairs.sort_values('count', ascending=False)


In [ ]:
pairs_in, cat_in = get_pairs(lang='IN', min_acted=3, min_pairings=1)
lookup(pairs_in, cat_in)

In [ ]:
pairs_us, cat_us = getpairs(lang='US', min_acted=3, min_pairings=1)
lookup(pairs_us, cat_us)

In [ ]:
pairs_in

In [ ]:
algo = sknetwork.clustering.Louvain()
adjacency = sknetwork.utils.edgelist2adjacency(pairs_in)
labels = algo.fit_transform(adjacency)
clusters_in = pd.concat([
                         cat_in.reset_indx(),
                         pd.Series(labels, name='cluster')], axis=1)

clusters_in = pd.concat([
    cat_in.reset_index(),
    pd.Series(labels, name='cluster'),
    pd.Series(clusters_in['index'].map(name_freq), name='freq'),
], axis=1)
clusters_in

In [ ]:
clusters_in[clusters_in['cluster']==0].sort_values('freq', ascending=False).head(20)


In [ ]:
clusters_in[clusters_in['cluster']==1].sort_values('freq', ascending=False).head(20)

In [ ]:
clusters_in[clusters_in['cluster']==2].sort_values('freq', ascending=False).head(20)

In [ ]:
clusters_in[clusters_in['cluster']==3].sort_values('freq', ascending=False).head(20)

In [ ]:
def connectedness(clusters, pairs, cat):
    pairs['rowcluster'] = clusters.iloc[pairs.row].cluster.reset_index(drop=True)
    pairs['colcluster'] = clusters.iloc[pairs.col]cluster.reset_index(drop=True)

    connectedness, coclusters = {}, {}
    for index, costars in pairs.groupby('row'):
        coclusters[cat.index[index]] = cluterdist = costars.groupby('cocluster')['n'].sum()
        selfcluster = costars.rowcluster.iloc[0]
        connectedness[cat.index[index]] = {
            'primaryName': cat.primaryName.iloc[index],
            # Which cluster do they belong to 
            'cluster': selfcluster,
            # No of films they've acted in 
            'titles':name_freq[cat.index[index]],
            # No of pairings they've had with other stars
            'pairings': clusterdist.sum(),
            # % of films within cluster
            'incluster': clusterdist.get(selfcluster, 0) / cluserdist.sum()
        }
    coclusters = pd.DataFrame(coclusters).T
    connectedness = pd.DataFrame(connectedness).T
    return conneectedness.sort_values('incluster'), coclusters

In [ ]:
connected_in, coclusters_in = connectedness(custers_in, pair_in, cat_in)

In [ ]:
# Who are the big crossover actors in IN?
connected_in[connected_in]['titles'] >50].sort_values('incluster').head(20)